In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score

test_df = pd.read_csv('Data/dataset_preprocessing.csv', index_col=0)
label = test_df["Survived"]
test_df = test_df.drop(["Survived"], axis=1)
test_df.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,0.271174,0.125,0.0,0.014151,0,0,1,0,1,0,0,0,1
1,0.472229,0.125,0.0,0.139136,1,0,0,1,0,1,0,0,0
2,0.321438,0.000,0.0,0.015469,0,0,1,1,0,0,0,0,1
3,0.434531,0.125,0.0,0.103644,1,0,0,1,0,0,0,0,1
4,0.434531,0.000,0.0,0.015713,0,0,1,0,1,0,0,0,1


### 1. Staging Model 로드 및 예측

In [7]:
# To do
# load staging model
model_uri = "models:/hellomlflow/Staging"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [8]:
loaded_model

RandomForestClassifier(max_depth=10, max_features=8, n_estimators=200)

In [9]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)

0.9214365881032548

### 2. Production Model 로드 및 예측

In [10]:
# To do
# load production model
model_uri = "models:/hellomlflow/Production"
loaded_model = mlflow.sklearn.load_model(model_uri)

c:\Users\hyunj\anaconda3\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


In [11]:
loaded_model

RandomForestClassifier(max_depth=10, max_features=5, n_estimators=50)

In [12]:
y_prd = loaded_model.predict(pd.DataFrame(test_df))
accuracy_score(y_prd, label)



0.9225589225589226

In [13]:
test_df.iloc[0].values

array([0.27117366, 0.125     , 0.        , 0.01415106, 0.        ,
       0.        , 1.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 1.        ])

### 3. MLFlow 를 이용한 API Serving 

In [ ]:
# mlflow models serve -m ./mlartifacts/679204153514078003/02e0f90282984a69a6185844b4c05f74/artifacts/model -p 5001 --no-conda


In [14]:
import pandas as pd
import requests

# Define local host and endpoint url
host = "127.0.0.1"
url = f"http://{host}:5001/invocations"

# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": test_df[:10].to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")



Pyfunc 'predict_interval':
${'predictions': [0, 1, 1, 1, 0, 0, 0, 0, 1, 1]}


In [15]:
# To do
# index 20~40 까지의 데이터 예측하기 
json_data = {"dataframe_split":test_df[20:40].to_dict(orient="split")}
response = requests.post(url, json=json_data)
print(f"\nPyfunc 'predict_interval':\n${response.json()}")


Pyfunc 'predict_interval':
${'predictions': [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]}


### 4. Cloud 배포 

##### AWS Sagemaker
* mlflow sagemaker build-and-push-container
* mlflow sagemaker deploy {parameters}

##### AzureML
* mlflow azureml export -m {model' path} -o test-output